In [1]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

from sqlalchemy import create_engine

In [2]:
gdf = ox.geometries_from_place('Austria', {'place':'city'}).to_crs("epsg:4326").reset_index(drop=True)[["name", "geometry"]]
gdf.head(7)

,name,geometry
0,Klagenfurt,POINT (14.30760 46.62394)
1,Linz,POINT (14.28620 48.30591)
2,Graz,POINT (15.43828 47.07087)
3,Wien,POINT (16.37250 48.20835)
4,Bratislava,"MULTIPOLYGON (((17.02860 48.19776, 17.02819 48..."
5,Salzburg,POINT (13.04648 47.79813)
6,Innsbruck,POINT (11.39277 47.26543)


In [3]:
gdf_tourism = []
for city in gdf['name']:
    if city == 'Wien':
        city = "Vienna"
    if city != "Bratislava":
        gdf_sub = ox.geometries_from_address(city+', Austria', {'tourism': 'museum'}).to_crs("epsg:4326").reset_index(drop=True)
        #
        cond = (gdf_sub["geometry"].geom_type == 'Polygon') | (gdf_sub["geometry"].geom_type == 'MultiPolygon')
        gdf_sub.loc[cond, "geometry"] = gdf_sub.loc[cond, "geometry"].to_crs("proj=+cea").centroid.to_crs("epsg:4326")
        #
        gdf_tourism.append(gdf_sub)
        print(city)
gdf_tourism = pd.concat(gdf_tourism)
gdf_tourism.head()

Klagenfurt
Linz
Graz
Vienna
Salzburg
Innsbruck


,name,tourism,wheelchair,wikipedia,geometry,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,...,building:part,roof:colour,roof:height,building:levels:underground,name:old,int_name,name:ar,addr:suburb,url,ref:isil
0,Thomas-Koschat-Museum,museum,no,de:Thomas_Koschat,POINT (14.31049 46.62032),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Museum Moderner Kunst Kärnten,museum,yes,de:Museum Moderner Kunst Kärnten,POINT (14.31013 46.62417),Klagenfurt am Wörthersee,AT,8,9021,Burggasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eboardmuseum,museum,yes,de:Eboardmuseum,POINT (14.30493 46.61649),Klagenfurt am Wörthersee,AT,20,9020,Florian-Gröger-Straße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Diözesanmuseum,museum,no,NaN,POINT (14.30938 46.62235),Klagenfurt am Wörthersee,AT,10/3,9020,Lidmanskygasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Landesarchiv,museum,NaN,NaN,POINT (14.30787 46.61824),Klagenfurt am Wörthersee,AT,7,NaN,St. Ruprechter Straße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
gdf_tourism

,name,tourism,wheelchair,wikipedia,geometry,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,...,building:part,roof:colour,roof:height,building:levels:underground,name:old,int_name,name:ar,addr:suburb,url,ref:isil
0,Thomas-Koschat-Museum,museum,no,de:Thomas_Koschat,POINT (14.31049 46.62032),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Museum Moderner Kunst Kärnten,museum,yes,de:Museum Moderner Kunst Kärnten,POINT (14.31013 46.62417),Klagenfurt am Wörthersee,AT,8,9021,Burggasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eboardmuseum,museum,yes,de:Eboardmuseum,POINT (14.30493 46.61649),Klagenfurt am Wörthersee,AT,20,9020,Florian-Gröger-Straße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Diözesanmuseum,museum,no,NaN,POINT (14.30938 46.62235),Klagenfurt am Wörthersee,AT,10/3,9020,Lidmanskygasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Landesarchiv,museum,NaN,NaN,POINT (14.30787 46.61824),Klagenfurt am Wörthersee,AT,7,NaN,St. Ruprechter Straße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,Audioversum,museum,yes,NaN,POINT (11.39678 47.26347),Innsbruck,AT,23,6020,Wilhelm-Greil-Straße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Galerie Mathias Mayr,museum,NaN,NaN,POINT (11.38853 47.26806),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AUT,museum,yes,NaN,POINT (11.39941 47.26105),Innsbruck,AT,1,6020,Lois-Welzenbacher-Platz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wilten,NaN,NaN
8,Landesmuseum Ferdinandeum,museum,NaN,de:Tiroler Landesmuseum,POINT (11.39773 47.26733),Innsbruck,AT,15,6020,Museumstraße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AT-FERD


In [5]:
gdf_tourism["geometry"].geom_type.unique()

array(['Point'], dtype=object)

In [6]:
host = "localhost"
database = "30DayMapChallenge2022"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [7]:
engine = create_engine(f"postgresql://{user}:{password}@{host}/{database}")  

In [8]:
gdf_tourism.to_postgis("museums_austria", engine, if_exists='replace')

In [9]:
gdf.to_postgis("city_austria", engine, if_exists='replace')